In [1]:
DEEPSEEK_API_KEY = 'sk-006151880202441eb4718f3f712fd48a'

# 1. 处理训练数据
## 1.1 数据读取

In [2]:
# 读取参考数据集
import json,time
from functools import reduce
novel_data = []
with open('./参考数据集.json', 'r',encoding='utf-8') as file:
    for line in file:
        novel_data.append(json.loads(line))

In [3]:
# 查看数据集中所有小说的名字和字数
for i in novel_data:
    print(f"《{i['name']}》的字数为： {len(i['text'])} 字")
    

《三国演义》的字数为： 593514 字
《水浒传》的字数为： 838774 字
《儒林外史》的字数为： 327064 字
《呼啸山庄》的字数为： 215931 字
《百年孤独》的字数为： 246912 字
《西游记》的字数为： 714847 字
《红与黑》的字数为： 361019 字
《战争与和平》的字数为： 1056718 字
《聊斋志异》的字数为： 384606 字
《醒世恒言》的字数为： 328581 字
《傲慢与偏见》的字数为： 218614 字
《红楼梦》的字数为： 789372 字
《隋唐演义》的字数为： 636063 字
《封神演义》的字数为： 588226 字
《拍案惊奇》的字数为： 238248 字
《尤利西斯》的字数为： 363155 字
《福尔摩斯探案集》的字数为： 363914 字


## 1.2 数据分块

In [4]:
# 安装必要的库
!pip install jieba loguru openai -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
     --------------------------------------- 0.0/19.2 MB 667.8 kB/s eta 0:00:29
     ---------------------------------------- 0.1/19.2 MB 1.1 MB/s eta 0:00:18
     ---------------------------------------- 0.2/19.2 MB 1.5 MB/s eta 0:00:13
      --------------------------------------- 0.3/19.2 MB 1.4 MB/s eta 0:00:14
      --------------------------------------- 0.3/19.2 MB 1.5 MB/s eta 0:00:13
      --------------------------------------- 0.5/19.2 MB 1.7 MB/s eta 0:00:12
     - -------------------------------------- 0.5/19.2 MB 1.8 MB/s eta 0:00:11
     - -------------------------------------- 0.6/19.2 MB 1.6 MB/s eta 0:00:12
     - -------------------------------------- 0.7/19.2 MB 1.7 MB/s eta 0:00:11
     - -------------------------------------- 0.8/19.2 MB 1.8 MB/s eta 0:00:11
     - -------------------------------------- 0.9/19.2 MB 1.8 MB/s eta 0:00:

## 1.3 大模型构建数据集

In [10]:
import json
from tqdm import tqdm
import time
import os
from openai import OpenAI

In [11]:
# 使用deepseek-chat api给段落打标签的接口
def get_response(text):
    client = OpenAI(
        api_key=DEEPSEEK_API_KEY,  # 如果您没有配置环境变量，请在此处用您的API Key进行替换
        base_url="https://api.deepseek.com",  # 填写DashScope SDK的base_url
    )
    completion = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {
                'role': 'system', 
                'content': '总结user提交的内容。用一句不超过50字的话总结这段小说的情节。仅回答总结，不需要添加其他内容。'
            },
            {
                'role': 'user', 
                'content': text
            }
        ])
    
    return completion.choices[0].message.content

In [12]:
# 设置容错机制，可最多重试 5 次，如果失败记录错误日志
def get_summary_with_retry(text):
    max_retries = 5
    retry_delay = 60  # in seconds
    attempts = 0
    while attempts < max_retries:
        try:
            return get_response(text)
        except Exception as e:
            attempts += 1
            if attempts < max_retries:
                logger.warning(f"Attempt {attempts} failed for text: {text}. Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                logger.error(f"All {max_retries} attempts failed for text: {text}. Error: {e}")
                raise

In [13]:
# 创建文件夹
os.makedirs('data', exist_ok=True)
os.makedirs('output', exist_ok=True)
os.makedirs('dataset', exist_ok=True)

In [14]:
# 批量给指定的小说打标签的接口函数
def build_dataset(novel,texts):
    instruction_prompt = "你是一个熟读各类小说的专家，请你根据要求写一段800字左右的小说。"
    dataset = []
    dataset_error = []
    for text in tqdm(texts, desc=f"Processing {novel}", total=len(texts)):
        try:
            summary = get_summary_with_retry(text)
            print(summary)
            dataset.append({
                "instruction": instruction_prompt,
                "input": summary,
                "output": text
            })
        except Exception as e:
            dataset_error.append(text)
            logger.error(f"Failed to process text: {text}. Error: {e}")
    
    with open(f"./data/{novel}.json", "w") as f:
        f.write(json.dumps(dataset, ensure_ascii=False, indent=4))

    with open(f"./data/{novel}_error.txt", "w") as f:
        f.write(json.dumps(dataset_error, ensure_ascii=False, indent=4))
    return dataset

In [22]:
# 拆分为 800 字一段的段落
import jieba

for i in range(len(novel_data)):
    data = novel_data[i]["text"]
    story_name = novel_data[i]["name"]
    print(len(data))

    # 利用jieba进行句子切分
    sentences = []

    for sentence in data.split('。'):  # 使用句号作为切分符
        sentences.append(sentence)

    # 将句子合并成800字一段的段落
    paragraphs = []
    current_paragraph = ''
    for sentence in sentences:
        if len(current_paragraph) + len(sentence) <= 800:
            current_paragraph += sentence+'。'
        else:
            paragraphs.append(current_paragraph.strip())
            current_paragraph = sentence

    # 将最后一段加入到段落列表中
    if current_paragraph:
        paragraphs.append(current_paragraph.strip())

    # 打印切分后的段落
    # for idx, paragraph in enumerate(paragraphs):
    #     print(f'段落 {idx + 1}: {paragraph}')
    print(len(paragraphs))
    # 开始给段落打标签
    dataset = build_dataset(story_name,paragraphs[:len(paragraphs)])

593514
758


Processing 三国演义:   0%|          | 1/758 [00:04<51:31,  4.08s/it]

小说讲述了汉朝末年，由于桓、灵二帝的昏庸和宦官专权，导致天下大乱，最终分裂为三国。张角得《太平要术》后，试图救世，但因宦官干政和天灾频发，天下更加混乱。


Processing 三国演义:   0%|          | 2/758 [00:08<55:11,  4.38s/it]

张角学习道术，自称“太平道人”，利用符水治病吸引信徒，建立势力，发动起义自称“天公将军”，引发大规模叛乱。朝廷紧急调兵镇压，同时幽州太守刘焉招募义兵应敌。涿县英雄刘备因此被引出，准备参与抗击叛军。


...

宋江受招安后，率军前往蓟州参拜罗真人，请求指点前程。罗真人接待宋江，并留他宿夜，讨论忠诚与未来。


Processing 水浒传:  74%|███████▍  | 800/1081 [58:08<21:26,  4.58s/it]

宋江向罗真人求问命运，罗真人预言宋江虽能善终，但一生多磨难，应适时退步，并赠予八句法语。宋江不解其意，罗真人表示此乃天机，未来自会明了。次日，罗真人允许公孙胜随宋江出征，并嘱咐宋江保重。


Processing 水浒传:  74%|███████▍  | 800/1081 [58:09<20:25,  4.36s/it]


KeyboardInterrupt: 

## 2.4 数据合并
由于采用多个小说数据进行模型微调，所以将data下的所有json数据文件合并成dataset文件夹下的`merged_story.json`文件

In [ ]:
import json
import os

# 设置文件夹路径
directory_path = './data'

# 初始化一个空列表，用于存储合并后的数据
merged_data = []

# 遍历文件夹下的所有文件
for filename in os.listdir(directory_path):
    # 检查文件扩展名是否为.json
    if filename.endswith('.json'):
        # 构建文件的完整路径
        file_path = os.path.join(directory_path, filename)
        # 打开并读取JSON文件
        with open(file_path, 'r', encoding='utf-8') as file:
            # 加载JSON内容到变量
            data = json.load(file)
            # 将当前文件的数据添加到合并列表中
            merged_data.extend(data)

# 将合并后的数据转换为JSON格式
merged_json = json.dumps(merged_data, ensure_ascii=False, indent=4)

# 可以选择将合并后的数据写入到一个新的JSON文件中
output_file_path = './dataset/merged_story.json'
with open(output_file_path, 'w', encoding='utf-8') as file:
    file.write(merged_json)

# 或者直接输出到控制台
# print(merged_json)

## 2.5 处理数据集

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [ ]:
# 将JSON文件转换为CSV文件
df = pd.read_json('./dataset/merged_story.json')
# df = pd.read_json('./data/story/呼啸山庄.json')
ds = Dataset.from_pandas(df)

In [ ]:
print(ds[:3])  # 查看一条数据
print(len(ds)) # 呼啸山庄.json, 总共84条微调指令数据, 如果采用merged_story.json，共9990条